In [1]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import random
import math
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFECV, RFE
import time

import matplotlib.pyplot as plt
import numpy as np
import math
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif, SelectKBest, SelectPercentile
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import time


def GenerateGaussianData(means, stds, datapoints):
    nrClasses = len(means)
    nrFeatures = len(means[0])
    Xdata = []
    Ydata = []
    for i in range(nrClasses):
        for j in range(datapoints[i]):
            features = []
            for k in range(nrFeatures):
                features.append(np.random.normal(
                    loc=means[i][k], scale=stds[i][k], size=None))
            Xdata.append(features)
            Ydata.append(i)
    Xdata = np.array(Xdata)
    Ydata = np.array(Ydata)
    Xdata, Ydata = shuffle(Xdata, Ydata)
    return Xdata, Ydata


def gaussian_clusters(nr_features=2, nr_classes=4, nr_data_points=10, \
                      random_diag=False, super_mega_random=False, variance_size=1, mean_variance=1, seed=12345):
    
    np.random.seed(seed)
    
    if super_mega_random:
        sigma = np.random.rand(nr_classes,nr_features)*variance_size
    elif random_diag:
        sigma = np.random.rand(nr_classes,nr_features)
        sigma[sigma > 0.5] = 1*variance_size
        sigma[sigma <= 0.5] = 0.01
    else:    
        sigma = [np.append([1]*(nr_features//2), [0.01]
                           * (nr_features-nr_features//2))]*nr_classes
    
    
    means = np.random.normal(0, mean_variance, (nr_classes, nr_features))
    dataPoints = [nr_data_points] * nr_classes
    Xdat, Ydat = GenerateGaussianData(means, sigma, dataPoints)
    return Xdat, Ydat




In [2]:
import matplotlib.pyplot as plt
import numpy as np
import math
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif, SelectKBest, SelectPercentile
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import time


def lda(x_filtered_train, y_filtered_train, x_filtered_test, y_filtered_test):

    lda_filtered_score = -1
    lda_unfiltered_score = -1
    # LDA classifier filtered data
    lda = LinearDiscriminantAnalysis()
    lda.fit(x_filtered_train, y_filtered_train)
    lda_filtered_score = lda.score(x_filtered_test, y_filtered_test)

    return lda_filtered_score


def knn(x_filtered_train, y_filtered_train, x_filtered_test, y_filtered_test):

    knn_filtered_score = -1
    knn_unfiltered_score = -1

    # Pruned data classifier
    neigh = KNeighborsClassifier()
    neigh.fit(x_filtered_train, y_filtered_train)
    knn_filtered_score = neigh.score(x_filtered_test, y_filtered_test)

    return knn_filtered_score


def run_filters(X_train, y_train, X_test, y_test):

    nr_features = X_train.shape[1]

    f_scores_filtered_knn = np.zeros(nr_features)
    f_scores_filtered_lda = np.zeros(nr_features)

    mi_scores_filtered_knn = np.zeros(nr_features)
    mi_scores_filtered_lda = np.zeros(nr_features)

    chi2_scores_filtered_knn = np.zeros(nr_features)
    chi2_scores_filtered_lda = np.zeros(nr_features)

    f_score_filter_times = np.zeros(nr_features)
    mi_filter_times = np.zeros(nr_features)
    chi2_filter_times = np.zeros(nr_features)

    f_score_knn_times = np.zeros(nr_features)
    mi_knn_times = np.zeros(nr_features)
    chi2_knn_times = np.zeros(nr_features)

    f_score_lda_times = np.zeros(nr_features)
    mi_lda_times = np.zeros(nr_features)
    chi2_lda_times = np.zeros(nr_features)

    for iK, k in enumerate(range(1, nr_features+1)):

        # fscore
        start = time.time()
        fscore = SelectKBest(f_classif, k=k)
        fscore.fit(X_train, y_train)
        end = time.time()
        f_score_filter_times[iK] = end - start

        # fscore knn
        start = time.time()
        f_knn_filtered_score = knn(
            fscore.transform(X_train), y_train, fscore.transform(X_test), y_test)
        end = time.time()
        f_score_knn_times[iK] = end - start

        # fscore lda
        start = time.time()
        f_lda_filtered_score = lda(
            fscore.transform(X_train), y_train, fscore.transform(X_test), y_test)
        end = time.time()
        f_score_lda_times[iK] = end - start

        f_scores_filtered_knn[iK] = f_knn_filtered_score
        f_scores_filtered_lda[iK] = f_lda_filtered_score

        # mutual_inf
        start = time.time()
        mi_score = SelectKBest(f_classif, k=k)
        mi_score.fit(X_train, y_train)
        end = time.time()
        mi_filter_times[iK] = end - start

        # mutal information knn
        start = time.time()
        mi_knn_filtered_score = knn(
            mi_score.transform(X_train), y_train, mi_score.transform(X_test), y_test)
        end = time.time()
        mi_knn_times[iK] = end - start

        # mutal information lda
        start = time.time()
        mi_lda_filtered_score = lda(
            mi_score.transform(X_train), y_train, mi_score.transform(X_test), y_test)
        end = time.time()
        mi_lda_times[iK] = end - start

        mi_scores_filtered_knn[iK] = mi_knn_filtered_score
        mi_scores_filtered_lda[iK] = mi_lda_filtered_score

        # Chi2
        scaler = MinMaxScaler()
        scaler.fit(X_train, y_train)
        # We need [0,1] for all values for chi2 to work
        X_train_normalized = scaler.transform(X_train)
        X_test_normalized = scaler.transform(X_test)
        start = time.time()
        X_chi2 = SelectKBest(chi2, k=k)
        X_chi2.fit(X_train_normalized, y_train)
        end = time.time()
        chi2_filter_times[iK] = end - start

        X_train_chi2 = X_chi2.transform(X_train_normalized)
        X_test_chi2 = X_chi2.transform(X_test_normalized)

        # Chi2 knn
        start = time.time()
        chi2_knn_filtered_score = knn(
            X_train_chi2, y_train, X_test_chi2, y_test)
        end = time.time()
        chi2_knn_times[iK] = end - start

        # Chi2 lda
        start = time.time()
        chi2_lda_filtered_score = lda(
            X_train_chi2, y_train, X_test_chi2, y_test)
        end = time.time()
        chi2_lda_times[iK] = end - start

        chi2_scores_filtered_knn[iK] = chi2_knn_filtered_score
        chi2_scores_filtered_lda[iK] = chi2_lda_filtered_score

    print("Order: f_score knn, mutual information knn, chi2 knn, f score lda, mutual information lda, chi2 lda")

    best_ks = np.argmax(f_scores_filtered_knn)+1, np.argmax(mi_scores_filtered_knn)+1, np.argmax(
        chi2_scores_filtered_knn)+1, np.argmax(f_scores_filtered_lda)+1, np.argmax(mi_scores_filtered_lda)+1, np.argmax(chi2_scores_filtered_lda)+1

    best_scores = np.max(f_scores_filtered_knn), np.max(mi_scores_filtered_knn), np.max(
        chi2_scores_filtered_knn), np.max(f_scores_filtered_lda), np.max(mi_scores_filtered_lda), np.max(chi2_scores_filtered_lda)

    times = np.sum(f_score_knn_times) + np.sum(f_score_filter_times), np.sum(mi_knn_times) + np.sum(mi_filter_times), np.sum(
        chi2_knn_times) + np.sum(chi2_filter_times), np.sum(f_score_lda_times)+np.sum(f_score_filter_times), np.sum(mi_lda_times) + np.sum(mi_filter_times), np.sum(chi2_lda_times) + np.sum(chi2_filter_times)

    return best_ks, best_scores, times


In [3]:
def vt_feat_sel(X_train, X_test, y_train, y_test,\
              class_alg=LinearDiscriminantAnalysis()):

    t_start = time.time()
    var = np.zeros(len(X_train[0,:]))

    for i in range(len(X_train[0,:])):

        var[i] = np.var(X_train[:,i])

        ind_max_var = np.argsort(var)

        vt_acc = np.zeros(len(X_train[0,:]))

        for i in range(len(X_train[0,:])):

            X_temp = X_train[:,ind_max_var[0:(i+1)]]
            X_test_temp = X_test[:,ind_max_var[0:(i+1)]]

            clf = LinearDiscriminantAnalysis()
            clf = clf.fit(X_temp, y_train)

            y_test_pred = clf.predict(X_test_temp)
            vt_acc[i] = accuracy_score(y_test, y_test_pred)

            vt_time = time.time()-t_start  
            vt_max_acc = max(vt_acc)
            vt_selected_features = np.argmax(vt_acc)

            return vt_time, vt_max_acc, vt_selected_features + 1


In [4]:
def rfe_feat_sel(X_train, X_test, y_train, y_test,\
                  class_alg=LinearDiscriminantAnalysis(), folds=5):

    t_start = time.time()
    # Train model
    rfecv = RFECV(class_alg, cv=folds)
    rfecv = rfecv.fit(X_train, y_train)

    rfe_time = time.time() - t_start

    scores = rfecv.grid_scores_[0:25]
    rfe_selected_features = np.argmax(scores)

    # Train model with optimal k
    rfe = RFE(class_alg,rfe_selected_features + 1)
    rfe = rfe.fit(X_train, y_train)

    y_test_pred = rfe.predict(X_test)
    rfe_acc = accuracy_score(y_test, y_test_pred)

    return rfe_time, rfe_acc, rfe_selected_features

In [5]:
def orig_class(X_train, X_test, y_train, y_test, class_alg=LinearDiscriminantAnalysis()):
  
    start_t = time.time()
    model = class_alg.fit(X_train,y_train)

    y_test_pred = model.predict(X_test)

    orig_acc = accuracy_score(y_test, y_test_pred)
    orig_time = time.time() - start_t

    return orig_time, orig_acc

In [6]:
from sklearn.model_selection import train_test_split
import numpy as np
import time
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score


def seq_feat_sel(X_train, X_test, y_train, y_test, num_feat = 30,\
                 forward_sfs=True, class_alg=LinearDiscriminantAnalysis(), folds=5):
    
    start = time.time()
    sfs = SFS(class_alg,
           k_features=(1, num_feat - 5),
           forward=forward_sfs,
           floating=False,
           verbose=0,
           scoring='accuracy',
           cv=folds)
    sfs = sfs.fit(X_train, y_train)
    end = time.time()
    sfs_time = end-start
    
    # Accuracy for test data
    wrap_feat_sfs = list(sfs.k_feature_idx_)
    class_alg.fit(X_train[:, wrap_feat_sfs], y_train)
    y_test_pred = class_alg.predict(X_test[:, wrap_feat_sfs])
    sfs_acc = accuracy_score(y_test, y_test_pred)
    
    return sfs_time, sfs_acc, len(wrap_feat_sfs)
  

def Plot_cv_train_acc_SFS(sfs, direc_str="forward"):
    plot_sfs(sfs.get_metric_dict(), kind='std_dev')
    plt.title('Sequential forward selection with std')
    plt.grid()

    plt.show()


nr_features=40
nr_classes=[3,5,10, 20, 30, 40]
nr_data_points = 1000
variance_size = 8
methods = ["Sequential forward selection", "Sequential backward selection", "RFE", \
           "f-score-knn", "Mutual information-knn", "Chi2-knn","f-score-lda", \
           "Mutual information-lda", "Chi2-lda", "Variance Threshold"]

accuracy_data = np.zeros([len(nr_classes), len(methods)])
time_data = np.zeros([len(nr_classes), len(methods)])
num_feat = np.zeros([len(nr_classes), len(methods)])
all_data_acc = np.zeros(len(nr_classes))
all_data_time = np.zeros(len(nr_classes))


for i in range(len(nr_classes)):
    print(i)

    X,y = gaussian_clusters(nr_features, nr_classes=nr_classes[i], \
                           nr_data_points=nr_data_points, random_diag=True, variance_size=variance_size)

    X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

    time_data[i, 0], accuracy_data[i, 0], num_feat[i, 0] = seq_feat_sel(X_train, X_test, y_train, y_test, forward_sfs=True)
    time_data[i, 1], accuracy_data[i, 1], num_feat[i, 1] = seq_feat_sel(X_train, X_test, y_train, y_test, forward_sfs=False)
    time_data[i, 2], accuracy_data[i, 2], num_feat[i, 2] = rfe_feat_sel(X_train, X_test, y_train, y_test)
    num_feat[i, 3:9], accuracy_data[i, 3:9], time_data[i, 3:9] = run_filters(X_train, y_train, X_test, y_test)
    time_data[i, 9], accuracy_data[i, 9], num_feat[i, 9] = vt_feat_sel(X_train, X_test, y_train, y_test)
    all_data_time[i], all_data_acc[i] = orig_class(X_train, X_test, y_train, y_test)
  

0


/usr/local/lib/python3.6/site-packages/sklearn/utils/__init__.py:93: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):
/usr/local/lib/python3.6/site-packages/sklearn/utils/__init__.py:93: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


Order: f_score knn, mutual information knn, chi2 knn, f score lda, mutual information lda, chi2 lda
1


/usr/local/lib/python3.6/site-packages/sklearn/utils/__init__.py:93: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):
/usr/local/lib/python3.6/site-packages/sklearn/utils/__init__.py:93: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


Order: f_score knn, mutual information knn, chi2 knn, f score lda, mutual information lda, chi2 lda
2


...

/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. 

/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. 

/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. 

/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/usr/local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. 

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
 
import numpy as np

print(accuracy_data[1,9])
print(all_data_acc[1])
accuracy_data_sub = (accuracy_data.T - all_data_acc.T).T
print(accuracy_data_sub.shape)

plt.figure(0)
nr_c = np.array([nr_classes])
df = pd.DataFrame(np.append(nr_c.T, np.append(accuracy_data_sub[:,0:3], \
                                              accuracy_data_sub[:,6:], axis=1), axis=1), columns=(["Num_classes"] + (methods[0:3]+methods[6:])))
df.plot(x="Num_classes", y=(methods[0:3]+methods[6:]), kind="bar")
plt.legend(loc='upper center', bbox_to_anchor=(1.45, 0.8), shadow=True, ncol=1)
plt.ylabel("accuracy")
plt.show()

plt.figure(1)
for i in range(len(methods)):
  plt.plot(nr_c.T, time_data[:, i], marker='.', label=methods[i])

plt.legend(loc='upper center', bbox_to_anchor=(1.45, 0.8), shadow=True, ncol=1)
plt.xlabel("Num classes")
plt.ylabel("time")
plt.show()